# Modelagem Preditiva (N2)

---

## 🗂️ [ Projeto ]

**Dataset:** [Nome do dataset ou fonte de dados]  
**Objetivo Geral:** [Breve explicação do propósito — ex.: “Aplicar técnicas de modelagem preditiva para identificar padrões e realizar previsões a partir dos dados tratados.”]  

---

## 🎯 Propósito do Notebook

Este notebook representa a **segunda etapa do pipeline analítico**, voltada à **modelagem preditiva** e **validação estatística**.  
A partir dos dados já preparados e consolidados no **N1 (Preparação de Dados)**, são aplicadas técnicas de **Machine Learning supervisionado** ou **não supervisionado**, de acordo com o tipo de problema em estudo (classificação, regressão ou agrupamento).

---

## 📋 Descrição Geral das Etapas

Nesta fase, o foco é **construir, testar e comparar modelos preditivos**, assegurando desempenho consistente e interpretabilidade.  
A execução segue as boas práticas de **ciência de dados reprodutível** e utiliza a **estrutura modular do Template de Dados**, garantindo clareza e padronização entre projetos.

**Tarefas previstas:**  
- Importação dos artefatos e metadados gerados no N1.  
- Separação entre **variáveis independentes (features)** e **variável-alvo (target)**, quando aplicável.  
- Divisão do dataset em **conjuntos de treino e teste**.  
- Construção de **pipelines de modelagem** com etapas de pré-processamento, ajuste e avaliação.  
- Treinamento de múltiplos algoritmos (ex.: *Regressão Logística, Random Forest, KNN, Árvores de Decisão, Regressão Linear, Clustering, etc.*).  
- Avaliação comparativa de desempenho com métricas adequadas (Acurácia, F1-Score, RMSE, Silhouette Score, entre outras).  
- Interpretação dos resultados e análise de importância das variáveis.  
- Exportação de modelos, métricas e relatórios para o **N3 (Análise e Insights)**.

---

> 💡 **Resumo:**  
> O N2 tem como objetivo transformar o dataset preparado no N1 em uma base analítica pronta para modelagem preditiva, explorando algoritmos, métricas e estratégias de validação.  
> É o núcleo experimental do projeto — onde hipóteses são testadas, modelos são ajustados e o conhecimento começa a emergir de forma quantitativa, sustentando a próxima etapa: o **N3 (Geração de Insights e Visualizações)**.


# 📘 Bootstrap do N2 — Inicialização do Ambiente e Configuração do Dataset

Esta etapa realiza o **bootstrap completo do N2**, responsável por preparar o ambiente,
carregar as configurações globais e disponibilizar o dataset processado para o início da modelagem supervisionada.

---

## ⚙️ Etapas executadas

1. **Localização automática da raiz do projeto (`PROJECT_ROOT`)**  
   O código percorre a árvore de diretórios até encontrar o arquivo `config/defaults.json`.  
   Isso garante que o notebook funcione corretamente mesmo quando aberto a partir de subpastas.

2. **Injeção da raiz e do módulo `utils/` no `sys.path`**  
   Essa etapa assegura que funções utilitárias, como as do arquivo `utils/utils_data.py`,
   possam ser importadas e utilizadas sem ajustes manuais de caminho.

3. **Carregamento das configurações globais (`defaults.json` + `local.json`)**  
   São lidas definições essenciais — como nome da variável alvo (`target_column`), 
   proporção de treino e teste (`test_size`), controle de aleatoriedade (`random_state`),
   estratégias de codificação, normalização e detecção de outliers.

4. **Validação da estrutura de diretórios padrão**  
   Pastas como `artifacts/`, `reports/` e `artifacts/models/` são verificadas e criadas automaticamente,  
   garantindo que o pipeline tenha onde salvar seus resultados e metadados.

5. **Descoberta e leitura do dataset processado**  
   O notebook identifica o arquivo final exportado pelo N1 (geralmente `data/processed/processed.parquet`),  
   realiza a leitura conforme o formato (Parquet, CSV ou Excel) e carrega o conteúdo em um `DataFrame` pandas.

6. **Identificação da variável alvo e separação de dados**  
   A coluna-alvo é localizada automaticamente com base nas configurações, e o conjunto é dividido em:  
   - `X`: features preditoras  
   - `y`: variável alvo (`TARGET_COL`)

7. **Verificações de sanidade e distribuição da target**  
   São exibidas métricas básicas como quantidade de linhas, colunas, valores nulos e distribuição da variável alvo.  
   Essas informações permitem validar rapidamente se o dataset está íntegro e pronto para o treinamento.

---

> 💡 **Em resumo:**  
> Esta célula garante que o N2 sempre inicie em um ambiente estável e configurado,  
> com o dataset processado e a variável-alvo corretamente carregados,  
> prontos para o pipeline de modelagem supervisionada.


In [7]:
# -*- coding: utf-8 -*-
from __future__ import annotations

from pathlib import Path
import sys
import importlib
import pandas as pd
from sklearn.model_selection import train_test_split


def _find_up(relative_path: str, start: Path | None = None) -> Path | None:
    start = start or Path.cwd()
    rel = Path(relative_path)
    for base in (start, *start.parents):
        cand = base / rel
        if cand.exists():
            return cand
    return None

_cfg = _find_up("config/defaults.json")
if _cfg is None:
    raise FileNotFoundError(
        "config/defaults.json não encontrado. Abra o notebook N2 dentro da estrutura do projeto."
    )

PROJECT_ROOT = _cfg.parent.parent.resolve()
UTILS_DIR = PROJECT_ROOT / "utils"
INIT_FILE = UTILS_DIR / "__init__.py"

if not UTILS_DIR.exists():
    raise ModuleNotFoundError(f"Pasta não encontrada: {UTILS_DIR}")
if not INIT_FILE.exists():
    INIT_FILE.write_text("", encoding="utf-8")

root_str = str(PROJECT_ROOT)
if root_str not in sys.path:
    sys.path.insert(0, root_str)

print(f"[INFO] PROJECT_ROOT: {PROJECT_ROOT}")
print(f"[INFO] sys.path ok. utils: {UTILS_DIR}")

import utils.utils_data as ud
importlib.reload(ud)  # garante versão mais recente se você editar utils_data.py


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] sys.path ok. utils: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\utils


<module 'utils.utils_data' from 'C:\\Users\\fabio\\Projetos DEV\\data projects\\telco-customer-churn-analysis\\utils\\utils_data.py'>

In [8]:
# =============================================================================
# N2 — Bootstrap do contexto, leitura do dataset e inicialização de X / y
# =============================================================================

import importlib
import utils.utils_data as ud
importlib.reload(ud)

# 1) Puxa o contexto completo (N2)
ctx = ud.n2_bootstrap_context()

# 2) Descompacta o que vai usar nas próximas células
PROJECT_ROOT   = ctx["project_root"]
cfg            = ctx["cfg"]
df             = ctx["df"]                 # dataframe completo, ainda com todas as colunas
X              = ctx["X"].copy()           # features já separadas no bootstrap
y              = ctx["y"].copy()           # target
TARGET_COL     = ctx["target_col"]
processed_path = ctx["processed_path"]

# Estes foram calculados a partir do X original (antes de qualquer remoção manual)
num_cols       = ctx["num_cols"]
cat_cols       = ctx["cat_cols"]
other_cols     = ctx["other_cols"]

test_size      = ctx["test_size"]
random_state   = ctx["random_state"]
scale_numeric  = ctx["scale_numeric"]

print(f"[INFO] Processed file: {processed_path}")
print(f"[INFO] Target: {TARGET_COL}")
print(f"[INFO] Shapes -> X: {X.shape} | y: {y.shape}")
print(f"[INFO] Colunas (antes de remoções manuais) -> num: {len(num_cols)} | cat: {len(cat_cols)} | other: {len(other_cols)}")

# 3) Painel estilizado N2 — visão geral do dataset processado (antes de remover colunas)
ud.n2_render_status_panel(ctx, keep_path_parts=4)


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] [ensure_dirs] artifacts=C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\artifacts | reports=C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports | models=C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\artifacts\models
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] Processed file: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\data\processed\processed.parquet
[INFO] Target: Churn
[INFO] Shapes -> X: (7043, 132) | y: (7043,)
[INFO] Colunas (antes de remoções manuais) -> num: 62 | cat: 71 | other: 0


# 🧹 Remoção de Colunas Irrelevantes para Modelagem

Nesta etapa removemos colunas que **não contribuem para o aprendizado supervisionado** e que podem até prejudicar o desempenho dos modelos.
O objetivo é garantir que o conjunto de features seja **enxuto, relevante e estável** antes de passar para o pré-processamento e construção dos pipelines de Machine Learning.

In [9]:
# =============================================================================
# Remoção de colunas irrelevantes para modelagem (IDs, flags derivadas, etc.)
# =============================================================================

cols_to_drop = ["customerID", "customerID_was_missing"]

X = X.drop(columns=cols_to_drop, errors="ignore")

# Recalcula num_cols, cat_cols e other_cols com base no X LIMPO
num_cols, cat_cols, other_cols = ud.summarize_columns(X)

print(f"[INFO] Colunas removidas de X: {cols_to_drop}")
print(f"[INFO] X agora possui {X.shape[1]} colunas.")
print(f"[INFO] Tipos de colunas após remoção -> num: {len(num_cols)} | cat: {len(cat_cols)} | other: {len(other_cols)}")


[INFO] Colunas removidas de X: ['customerID', 'customerID_was_missing']
[INFO] X agora possui 130 colunas.
[INFO] Tipos de colunas após remoção -> num: 61 | cat: 69 | other: 0


# ✂️ Split Treino/Teste e Resumo de Colunas

Nesta etapa, preparamos os dados para a modelagem supervisionada, separando variáveis independentes (`X`) da variável alvo (`y`) e realizando o **split treino/teste** com parâmetros consistentes definidos no `defaults.json`.

## O que acontece aqui

1. **Separação de X e y**
   - Remove-se a coluna alvo (`TARGET_COL`) de `X` e mantém-se `y` intacta para preservar a distribuição original.

2. **Resumo de tipos de colunas**
   - Gera contagens de colunas **numéricas**, **categóricas** e **outras** (se houver).
   - Este diagnóstico orienta a construção do `ColumnTransformer` na próxima etapa.

3. **Split treino/teste com estratificação**
   - Usa `test_size` e `random_state` da configuração.
   - Ativa `stratify=y` sempre que houver mais de uma classe, mantendo a **mesma proporção** de classes em `train` e `test`.

4. **Diagnósticos úteis**
   - Compara a **distribuição da target** no conjunto geral vs. `train` vs. `test` (contagem e %).
   - Sinaliza **desbalanceamento** quando a classe majoritária ultrapassa um limite (ex.: 80%).
   - Detecta **categorias raras** (freq. < 5 no `train`) em variáveis categóricas — útil para evitar explosão do One-Hot e para tratar identificadores.

## Observações frequentes

- **IDs como `customerID`** aparecem como inúmeras categorias raras. Em geral, **não devem ser usadas como preditores**, pois são identificadores sem relação causal. Recomenda-se **removê-las** de `X` (ou excluí-las da lista de categóricas antes do One-Hot).
- Se houver **alto cardinalidade** e a variável for preditiva, considere técnicas específicas (target encoding, hashing, catboost encoder). Neste template, mantemos a abordagem pedagógica e transparente com One-Hot e remoção de IDs.

## Próximo passo

- Construir o **pré-processamento** com `ColumnTransformer` (imputação + One-Hot para categóricas + escala opcional para numéricas) e ajustar no conjunto de treino.


In [10]:
# =============================================================================
# Split Treino/Teste e Resumo de Colunas
# =============================================================================

import importlib
import utils.utils_data as ud
importlib.reload(ud)

from sklearn.model_selection import train_test_split

# 🔹 IMPORTANTE:
# Aqui NÃO recriamos X e y a partir de df.
# Usamos o X e y que já foram preparados nas células anteriores
# (com remoção de customerID e outras colunas irrelevantes).

# Se quiser garantir, pode imprimir:
print(f"[INFO] X (após remoção) -> shape: {X.shape}")
print(f"[INFO] y -> shape: {y.shape}")

# Se você recalculou num_cols, cat_cols, other_cols na etapa de remoção,
# eles já estão coerentes com o X atual.
print(f"[INFO] Colunas numéricas: {len(num_cols)} | "
      f"categóricas: {len(cat_cols)} | ignoradas: {len(other_cols)}")

# Parâmetros do split vindos do cfg (estrutura FLAT atual)
split_params = {
    "test_size": cfg.get("test_size", 0.2),
    "random_state": cfg.get("random_state", 42),
    "stratify": True,  # mantém o comportamento atual
}

test_size = split_params["test_size"]
random_state = split_params["random_state"]
do_stratify = split_params["stratify"]

print(f"[INFO] Split params -> test_size={test_size} | "
      f"random_state={random_state} | stratify={do_stratify}")

# Split com estratificação quando aplicável
stratify_vec = y if do_stratify else None

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=test_size,
    random_state=random_state,
    stratify=stratify_vec,
)

print(f"[INFO] X_train: {X_train.shape} | X_test: {X_test.shape}")

# -----------------------------------------------------------------------------
# Detecção simples de categorias raras nas colunas categóricas (no train)
# -----------------------------------------------------------------------------
rare_threshold = 5  # pode puxar isso de config depois, se quiser
rare_categories_info = {}

for col in cat_cols:
    vc = X_train[col].value_counts(dropna=False)
    n_rare = (vc < rare_threshold).sum()
    if n_rare > 0:
        rare_categories_info[col] = int(n_rare)

if rare_categories_info:
    print(f"[INFO] Colunas categóricas com categorias raras (<{rare_threshold} amostras) no train:")
    for col, n_rare in rare_categories_info.items():
        print(f" - {col}: {n_rare} categorias raras")
else:
    print("[INFO] Nenhuma categoria rara detectada no train com o limite atual.")

# -----------------------------------------------------------------------------
# Painel estilizado (HTML) centralizado no utils_data.py
# -----------------------------------------------------------------------------
ud.n2_display_split_and_column_summary(
    numeric_cols=num_cols,
    categorical_cols=cat_cols,
    ignored_cols=other_cols,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    target_name=TARGET_COL,
    split_params=split_params,
    rare_categories=rare_categories_info,
    rare_threshold=rare_threshold,
    logger=None,
)


[INFO] X (após remoção) -> shape: (7043, 130)
[INFO] y -> shape: (7043,)
[INFO] Colunas numéricas: 61 | categóricas: 69 | ignoradas: 0
[INFO] Split params -> test_size=0.2 | random_state=42 | stratify=True
[INFO] X_train: (5634, 130) | X_test: (1409, 130)
[INFO] Nenhuma categoria rara detectada no train com o limite atual.


Churn,count,pct
no,5174,73.46
yes,1869,26.54
Churn,count,pct
no,4139,73.46
yes,1495,26.54
Churn,count,pct
no,1035,73.46
yes,374,26.54


# ⚙️ Pré-processamento — One-Hot denso + escala opcional

Nesta etapa, preparamos `X` para treinamento aplicando **imputação**, **codificação categórica** e **padronização opcional**.  
Mantemos este bloco **visível no N2** por transparência pedagógica: assim, qualquer pessoa que leia o notebook consegue entender claramente *como* os dados chegam ao modelo.

Esta etapa utiliza diretamente:
- as colunas já filtradas na **remoção de variáveis irrelevantes**,  
- e o resultado do **Split Treino/Teste e Resumo de Colunas**.

Ou seja: aqui entra apenas o que **realmente será aprendido pelo modelo**.

---

## 🔧 O que acontece aqui

### **1. Identificação dos grupos de colunas**
Usamos `num_cols` e `cat_cols` (derivados de `summarize_columns(X)`) para aplicar transformações adequadas a cada tipo de dado.

---

### **2. Pipeline numérico**
Inclui:

- `SimpleImputer(strategy="mean")` → preenche valores ausentes com a média  
- `StandardScaler()` → **opcional**, definido por `cfg["scale_numeric"]`

A montagem é dinâmica:  
- se `scale_numeric=True`, o scaler é incluído;  
- se `False`, fica somente o imputador.

---

### **3. Pipeline categórico**
Responsável pela expansão de categorias:

- `SimpleImputer(strategy="most_frequent")`  
- `OneHotEncoder(handle_unknown="ignore", saída densa)`

O Encoder é montado de forma **compatível com múltiplas versões do scikit-learn**, usando:

- `sparse_output=False` em versões novas  
- `sparse=False` em versões antigas  

Isso garante que o template funcione em diferentes ambientes.

---

### **4. ColumnTransformer**
Reúne todos os pipelines:

- Aplica as transformações **somente** às colunas especificadas  
- Descarta o restante com `remainder="drop"`  
- Mantém nomes limpos com `verbose_feature_names_out=False`  

O resultado é uma matriz numérica pronta para alimentar qualquer modelo tradicional.

---

### **5. Ajuste e transformação**
Realizamos:

- `fit(X_train, y_train)` → aprende imputações e escalas no *treino*  
- `transform(X_train/X_test)` → aplica transformações consistentes  
- Converte o resultado para **matriz densa**, caso venha esparso de algum backend

---

### **6. Diagnósticos rápidos**
Após a transformação, exibimos:

- estado de `scale_numeric`  
- contagem de colunas numéricas e categóricas  
- número total de features geradas  
- *preview* das primeiras features  
- shapes finais de `X_train_t` e `X_test_t`  
- consumo de memória em MB  

Tudo isso aparece num **card visual padronizado**, mantendo consistência com as etapas anteriores.

---

## ⚠️ Observações importantes

- **Identificadores (IDs)** como `customerID` devem ser removidos *antes* desta etapa — caso contrário, geram explosão do One-Hot sem fornecer qualquer valor preditivo.
- Colunas categóricas com **alta cardinalidade** podem aumentar muito o número de features. Alternativas para projetos futuros:
  - hashing trick  
  - target encoding  
  - CatBoost encoding  
- Se a memória ficar alta, considere:
  - agrupar categorias raras  
  - reduzir cardinalidade  
  - usar One-Hot esparso + modelos compatíveis com matrizes esparsas  

In [6]:
# =============================================================================
# Pré-processamento — One-Hot denso + escala opcional
# =============================================================================

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd


# -----------------------------------------------------------------------------
# Função compatível com versões antigas/novas do scikit-learn
# -----------------------------------------------------------------------------
def build_preprocess(numeric_cols, categorical_cols, scale_numeric=True):
    """
    Cria um ColumnTransformer com:
      - Numéricas: imputação média + (opcional) StandardScaler
      - Categóricas: imputação mais frequente + OneHotEncoder denso (compatível)
    """

    # OneHotEncoder compatível com sklearn >= 1.4 e versões anteriores
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

    # Pipeline numérico
    num_steps = [("imputer", SimpleImputer(strategy="mean"))]
    if scale_numeric and len(numeric_cols) > 0:
        num_steps.append(("scaler", StandardScaler()))
    numeric_transformer = Pipeline(steps=num_steps)

    # Pipeline categórico
    cat_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", ohe),
    ])

    # ColumnTransformer final
    ct = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_cols if len(numeric_cols) > 0 else []),
            ("cat", cat_transformer, categorical_cols if len(categorical_cols) > 0 else []),
        ],
        remainder="drop",
        verbose_feature_names_out=False,
    )

    return ct


# -----------------------------------------------------------------------------
# Construção do pré-processador conforme config
# -----------------------------------------------------------------------------
scale_numeric = bool(cfg.get("scale_numeric", True))
preprocess = build_preprocess(num_cols, cat_cols, scale_numeric=scale_numeric)

print(f"[INFO] scale_numeric={scale_numeric} | num_cols={len(num_cols)} | cat_cols={len(cat_cols)}")

# -----------------------------------------------------------------------------
# Ajuste (fit) no conjunto de treino
# -----------------------------------------------------------------------------
preprocess.fit(X_train, y_train)

# -----------------------------------------------------------------------------
# Inspeção opcional de nomes das features geradas
# -----------------------------------------------------------------------------
feat_names = None
try:
    feat_names = preprocess.get_feature_names_out()
    print(f"[INFO] Features transformadas: {len(feat_names)}")
except Exception as e:
    print(f"[WARN] Não foi possível obter nomes de features: {e}")

# -----------------------------------------------------------------------------
# Transformar treino e teste (arrays densos)
# -----------------------------------------------------------------------------
X_train_t = preprocess.transform(X_train)
X_test_t  = preprocess.transform(X_test)

if hasattr(X_train_t, "toarray"):
    X_train_t = X_train_t.toarray()
if hasattr(X_test_t, "toarray"):
    X_test_t = X_test_t.toarray()

print(f"[INFO] Shapes transformados -> X_train_t: {X_train_t.shape} | X_test_t: {X_test_t.shape}")

# -----------------------------------------------------------------------------
# Memória estimada
# -----------------------------------------------------------------------------
def _mb(bytecount):
    return (bytecount or 0) / (1024**2)

mem_train = X_train_t.nbytes if isinstance(X_train_t, np.ndarray) else None
mem_test  = X_test_t.nbytes  if isinstance(X_test_t,  np.ndarray) else None

print(f"[INFO] Memória estimada -> train={_mb(mem_train):.2f} MB | test={_mb(mem_test):.2f} MB")

# -----------------------------------------------------------------------------
# Painel estilizado (HTML) no padrão N2
# -----------------------------------------------------------------------------
ud.n2_display_preprocess_summary(
    num_cols=num_cols,
    cat_cols=cat_cols,
    scale_numeric=scale_numeric,
    X_train_t=X_train_t,
    X_test_t=X_test_t,
    feat_names=feat_names,
    mem_train=mem_train,
    mem_test=mem_test,
)


[INFO] scale_numeric=True | num_cols=61 | cat_cols=69
[INFO] Features transformadas: 151
[INFO] Shapes transformados -> X_train_t: (5634, 151) | X_test_t: (1409, 151)
[INFO] Memória estimada -> train=6.49 MB | test=1.62 MB


feature 1,feature 2,feature 3
SeniorCitizen,tenure,MonthlyCharges
TotalCharges,TotalCharges_was_missing,charge_gap
charge_gap_log1p,SeniorCitizen_was_missing,tenure_was_missing
MonthlyCharges_was_missing,charge_gap_was_missing,charge_gap_log1p_was_missing
gender_was_missing,Partner_was_missing,Dependents_was_missing
PhoneService_was_missing,MultipleLines_was_missing,InternetService_was_missing
OnlineSecurity_was_missing,OnlineBackup_was_missing,DeviceProtection_was_missing
TechSupport_was_missing,StreamingTV_was_missing,StreamingMovies_was_missing


## 4) Seletor de modelos e hiperparâmetros (UI)

In [11]:
# === UI: Seletor de modelos + Hyperdrive (encapsulado no utils_data) ===
from utils.utils_data import n2_build_models_ui

n2_build_models_ui(
    preprocess=preprocess,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    models_dir=models_dir,
    reports_dir=reports_dir
)


Box(children=(VBox(children=(HBox(children=(HTML(value="<div class='lumen-title'>Seletor de modelos · Hyperdri…

## 5) Treino, avaliação e export de artefatos

In [ ]:

def collect_params_from_tab():
    return {name: {k: w.value for k, w in spec["params"].items()} for name, spec in MODEL_REGISTRY.items()}

def compute_and_plot(pipe, name, X_test, y_test):
    y_pred = pipe.predict(X_test)
    metrics = compute_metrics(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(5,4))
    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred)).plot(ax=ax, colorbar=False)
    ax.set_title(f"Matriz de confusão — {name}")
    plt.show()
    try_plot_roc(pipe, X_test, y_test)
    print(f"[OK] {name}: accuracy={metrics['accuracy']:.4f} | f1={metrics['f1']:.4f}")
    return metrics

def train_and_eval(models_selected, params_by_model):
    results = {}
    for name, selected in models_selected.items():
        if not selected: 
            continue
        ModelClass = MODEL_REGISTRY[name]["class"]
        params = params_by_model.get(name, {})
        pipe = Pipeline(steps=[("preprocess", preprocess), ("clf", ModelClass(**params))])
        pipe.fit(X_train, y_train)
        metrics = compute_and_plot(pipe, name, X_test, y_test)
        results[name] = {"pipeline": pipe, "metrics": metrics, "params": params}
    return results

@out.capture()
def on_train_clicked(_):
    clear_output(wait=True)
    display(W.HTML("<h4>Treinando...</h4>"))
    selected = {name: chk.value for name, chk in model_checks.items()}
    params = collect_params_from_tab()
    results = train_and_eval(selected, params)

    if results:
        df_rank = pd.DataFrame([{"model": k, **v["metrics"], **{"params": v["params"]}} for k, v in results.items()])\
                    .sort_values(by=["f1", "accuracy"], ascending=False)
        display(W.HTML("<h4>Ranking (F1, depois Accuracy)</h4>")); display(df_rank)
        if cb_persist.value:
            for name, rec in results.items():
                persist_artifacts(name, rec["pipeline"], rec["metrics"], rec["params"], models_dir, reports_dir)
    else:
        print("[AVISO] Nenhum modelo selecionado.")

btn_train.on_click(on_train_clicked)
